# Create Configs with all Combinations of Given Value Lists

## Default Values for Papermill Parameters

In [ ]:
PARAM_BASE_CONFIG_FILENAME = "../outputs/base_config.yaml"
PARAM_CONFIG_VARIABLES_FILENAME = "../outputs/config_variables.yaml"
PARAM_CONFIG_VARIABLES_COUPLINGS_FILENAME = "../outputs/config_variables_couplings.yaml"
PARAM_CONFIG_BASENAME_PREFIXES_FILENAME = "../outputs/config_basename_prefixes.yaml"
PARAM_CONFIG_BASENAME_SUFFIXES_FILENAME = "../outputs/config_basename_suffixes.yaml"
PARAM_CONFIG_OUT_PATH = "../outputs"
PARAM_CONFIG_OUT_FILENAME_PREFIX = None

## Import and Set Parameters

In [ ]:
from subroc import util

import yaml
import itertools
import numpy as np
import os

if PARAM_CONFIG_OUT_FILENAME_PREFIX is None:
    PARAM_CONFIG_OUT_FILENAME_PREFIX = ""

# fill environment variables into params
PARAM_BASE_CONFIG_FILENAME = util.prepend_experiment_definition_path(PARAM_BASE_CONFIG_FILENAME)
PARAM_CONFIG_VARIABLES_FILENAME = util.prepend_experiment_definition_path(PARAM_CONFIG_VARIABLES_FILENAME)
PARAM_CONFIG_VARIABLES_COUPLINGS_FILENAME = util.prepend_experiment_definition_path(PARAM_CONFIG_VARIABLES_COUPLINGS_FILENAME)
PARAM_CONFIG_BASENAME_PREFIXES_FILENAME = util.prepend_experiment_definition_path(PARAM_CONFIG_BASENAME_PREFIXES_FILENAME)
PARAM_CONFIG_BASENAME_SUFFIXES_FILENAME = util.prepend_experiment_definition_path(PARAM_CONFIG_BASENAME_SUFFIXES_FILENAME)
PARAM_CONFIG_OUT_PATH = util.prepend_experiment_definition_path(PARAM_CONFIG_OUT_PATH)

STAGE_OUTPUT_PATH = os.environ.get("STAGE_OUTPUT_PATH", "../outputs")

## Read Base Config

In [ ]:
with open(PARAM_BASE_CONFIG_FILENAME) as base_config_file:
    base_config = yaml.load(base_config_file, Loader=yaml.FullLoader)

print("base_config =", base_config)

## Read Config Variables

In [ ]:
with open(PARAM_CONFIG_VARIABLES_FILENAME) as config_variables_file:
    config_variables = yaml.load(config_variables_file, Loader=yaml.FullLoader)

print("config_variables =", config_variables)

## Read Config Variables Couplings

In [ ]:
with open(PARAM_CONFIG_VARIABLES_COUPLINGS_FILENAME) as config_variables_file:
    config_variables_couplings = yaml.load(config_variables_file, Loader=yaml.FullLoader)

if config_variables_couplings is None:
    config_variables_couplings = []

print("config_variables_couplings =", config_variables_couplings)

## Prepare Parameter Data Structure Including Couplings
### compute list of lists of dicts for the couplings: inner lists contain coupled parameters with available values

In [ ]:
variables_all_parameters = []
all_coupled_variables = []

# add the explicitly coupled parameters
for coupled_parameters in config_variables_couplings:
    all_coupled_variables += coupled_parameters
    coupled_parameter_value_dicts = []

    for coupled_value_index in range(len(config_variables[coupled_parameters[0]])):
        coupled_parameter_value_dict = {}

        for parameter in coupled_parameters:
            coupled_parameter_value_dict[parameter] = config_variables[parameter][coupled_value_index]
        
        coupled_parameter_value_dicts.append(coupled_parameter_value_dict)
    
    variables_all_parameters.append(coupled_parameter_value_dicts)

# add remaining parameters one by one to represent that they are not coupled
for parameter in config_variables.keys():
    if parameter in all_coupled_variables:
        continue

    parameter_value_dicts = []

    for parameter_value in config_variables[parameter]:
        parameter_value_dicts.append({parameter: parameter_value})
    
    variables_all_parameters.append(parameter_value_dicts)

print(variables_all_parameters)
print("number of combinations:", "*".join([str(len(value_dicts)) for value_dicts in variables_all_parameters]), "=", np.prod([len(value_dicts) for value_dicts in variables_all_parameters]))

## Read Config Basename Prefixes

In [ ]:
with open(PARAM_CONFIG_BASENAME_PREFIXES_FILENAME) as config_basename_prefixes_file:
    config_basename_prefixes = yaml.load(config_basename_prefixes_file, Loader=yaml.FullLoader)

print("config_basename_prefixes =", config_basename_prefixes)

## Read Config Basename Suffixes

In [ ]:
with open(PARAM_CONFIG_BASENAME_SUFFIXES_FILENAME) as config_basename_suffixes_file:
    config_basename_suffixes = yaml.load(config_basename_suffixes_file, Loader=yaml.FullLoader)

print("config_basename_suffixes =", config_basename_suffixes)

## Merge Prefixes and Suffixes to a Single Data Structure

In [ ]:
basename_pre_and_suffixes = {}

for key, item in config_basename_prefixes.items():
    basename_pre_and_suffixes[key] = {"pre": item, "suf": ""}

for key, item in config_basename_suffixes.items():
    current = basename_pre_and_suffixes[key]

    if current is None:
        current = {"pre": ""}
    
    current["suf"] = item

    basename_pre_and_suffixes[key] = current

## Write Config for Each Parameter Value Combination

In [ ]:
# use variables_all_parameters in itertools.product to get value combinations that respect the couplings
for value_dicts in itertools.product(*variables_all_parameters):
    combined_values_dict = {}

    # extract parameter name-value pairs from the inner dicts for each value combination
    for value_dict in value_dicts:
        combined_values_dict |= value_dict

    # add the current value combination to the base config
    base_config |= combined_values_dict
    
    value_combination_string = "_".join([str(v) for v in combined_values_dict.values()])
    
    # add combination-based filenames to the base config (e.g. for output files)
    for key, pre_and_suf in basename_pre_and_suffixes.items():
        base_config[key] = pre_and_suf["pre"] + value_combination_string + pre_and_suf["suf"]

    print(base_config)
    
    # save the complete config for this parameter value combination
    # for the next stage
    with open(f"{PARAM_CONFIG_OUT_PATH}/{PARAM_CONFIG_OUT_FILENAME_PREFIX}{value_combination_string}.yaml", "w") as outfile:
        yaml.dump(base_config, outfile)

    # for debugging also into the output dir
    with open(f"{STAGE_OUTPUT_PATH}/{PARAM_CONFIG_OUT_FILENAME_PREFIX}{value_combination_string}.yaml", "w") as outfile:
        yaml.dump(base_config, outfile)